# **LSTM**
## 이론
![[image/Pasted image 20251022141148.png]]
> 입출력 기록에 대한 부분을 보완함(RNN의 단점 극복을 위함)
### 1) LSTM(Long Short-Term Memory)

- **문제 배경**: 단순 RNN은 긴 문장에서 **기울기 소실**로 초반 정보가 사라지기 쉬움.
- **핵심 아이디어**: “**셀 상태(cell state)**”라는 **고속도로**를 유지하면서 필요한 정보만 게이트로 **추가·제거**.
- **세 가지 게이트**
    - **망각 게이트** (f_t): 무엇을 **버릴지** 결정
    - **입력 게이트** (i_t): 무엇을 **새로 담을지** 결정 (후보 ( \tilde{C}_t ))
    - **출력 게이트** (o_t): 어떤 **은닉 상태**를 다음으로 보낼지 결정
- **효과**: 먼 과거의 신호를 **오래 보존**하면서도, 불필요한 정보는 버릴 수 있음 → 장기 의존성 해결.

### 2) Bidirectional(양방향) RNN/LSTM
![[image/Pasted image 20251022141344.png]]
- **아이디어**: 문장을 **왼→오(정방향)**, **오→왼(역방향)**으로 **동시에** 읽음.
- **효과**: 현재 단어 예측 시 **앞·뒤 문맥** 모두 활용 → 의존성이 양방향인 태스크(감성·개체명 인식 등)에서 유리.
- **주의**: 실시간 스트리밍처럼 미래를 볼 수 없는 경우에는 적용 불가.

### 3) Dropout (과적합 완화)
![[image/Pasted image 20251022141445.png]]
> 너무 많은 가중치가 연산을 방해할 수 있음 -> 중요하지 않은 뉴런을 죽임
> -> 효율성 향상/과적합 대응력 향상/학습속도 향상


- **일반 Dropout**: 레이어 출력을 학습 중 확률적으로 **0**으로 만들어 **공동 적응(co-adaptation)** 을 방지.
- **Recurrent Dropout**: 순환(은닉→은닉) 연결에도 드롭아웃을 적용.
- **Keras 적용 위치**
    - `Dropout(p)`: Embedding 뒤, LSTM 뒤 등 **레이어 사이**
    - `LSTM(..., dropout=p, recurrent_dropout=q)`: LSTM 내부 입력·순환 연결에 직접 적용
- **팁**: `recurrent_dropout`은 GPU에서 느릴 수 있음. 처음에는 **외부 Dropout**부터 적용 → 필요 시 `recurrent_dropout` 소량(0.1~0.2).

In [5]:
# ======================
# 0) 환경
# ======================
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

np.random.seed(42)
tf.random.set_seed(42)

# ======================
# 1) 미니 데이터
# ======================
texts = [
    "이 영화 정말 재미있다", "배우 연기가 훌륭하다", "감동적인 스토리에 눈물이 났다", "완전 추천한다 최고다",
    "음악과 영상미가 너무 좋았다", "유머가 자연스럽고 몰입됐다", "따뜻하고 여운이 긴 작품", "감독의 연출이 인상적이다",
    "최악이다 돈이 아깝다", "지루하고 시간 낭비였다", "스토리가 엉성하고 별로다", "다시는 보고 싶지 않다",
    "캐릭터가 매력 없고 산만했다", "전개가 느리고 답답했다", "웃음 포인트가 전혀 없었다", "실망스러운 마무리였다"
]
labels = [1,1,1,1, 1,1,1,1, 0,0,0,0, 0,0,0,0]

# ======================
# 2) 토크나이징 & 패딩
# ======================
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

vocab_size = 2000
maxlen = 10

tok = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tok.fit_on_texts(texts)

# 문장 → 시퀀스 → 패딩
X = pad_sequences(tok.texts_to_sequences(texts), maxlen=maxlen, padding="post")
y = np.array(labels)

# ======================
# 3) 데이터 분리 (train/test)
# ======================
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print("훈련 데이터 크기:", X_train.shape)
print("검증 데이터 크기:", X_val.shape)

# ======================
# 3) 모델 빌더
# ======================
embedding_dim = 32
units = 32

# --- (1) 기본 LSTM 모델 ---
def build_base():
    m = Sequential([
        Embedding(vocab_size, embedding_dim),   # input_length 제거 (자동 인식)
        LSTM(units),
        Dense(1, activation="sigmoid")
    ])
    m.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return m

# --- (2) Dropout 추가 LSTM ---
def build_dropout():
    m = Sequential([
        Embedding(vocab_size, embedding_dim),
        Dropout(0.2),                 # Embedding 출력 정규화
        LSTM(units, dropout=0.2),     # 내부 입력 드롭아웃
        Dropout(0.2),                 # LSTM 출력 정규화
        Dense(1, activation="sigmoid")
    ])
    m.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return m

# --- (3) 양방향 LSTM ---
def build_bilstm():
    m = Sequential([
        Embedding(vocab_size, embedding_dim),
        Bidirectional(LSTM(units)),   # 순방향 + 역방향 학습
        Dense(1, activation="sigmoid")
    ])
    m.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return m


# ======================
# 4) 학습 & 리포트
# ======================
def train(name, model):
    """
    모델을 학습시키고, 마지막 epoch의 train/val 정확도를 출력
    """
    h = model.fit(
        X_train, y_train,             # ✅ 변수명 수정됨
        epochs=12, batch_size=4,
        validation_data=(X_val, y_val),
        verbose=0
    )
    print(f"{name:>16} | Train={h.history['accuracy'][-1]:.3f} | Val={h.history['val_accuracy'][-1]:.3f}")
    return model


print("✅ 모델별 성능(소형 데이터, 참고용)")
m_base   = train("Base LSTM",        build_base())
m_drop   = train("LSTM + Dropout",   build_dropout())
m_bilstm = train("BiLSTM",           build_bilstm())


# ======================
# 5) 샘플 예측
# ======================
samples = ["정말 감동적이고 훌륭한 영화", "지루하고 별로였어 다시 안봐"]

# 문장 → 시퀀스 → 패딩
pad_s = pad_sequences(tok.texts_to_sequences(samples), maxlen=maxlen, padding="post")

# 모델별 예측 결과 확인
for m, tag in [(m_base, "Base"), (m_drop, "Dropout"), (m_bilstm, "BiLSTM")]:
    p = (m.predict(pad_s, verbose=0) > 0.5).astype(int).ravel()
    print(f"{tag:>7} 예측:", list(zip(samples, p)))


훈련 데이터 크기: (12, 10)
검증 데이터 크기: (4, 10)
✅ 모델별 성능(소형 데이터, 참고용)
       Base LSTM | Train=1.000 | Val=0.500
  LSTM + Dropout | Train=0.750 | Val=0.500
          BiLSTM | Train=1.000 | Val=0.250
   Base 예측: [('정말 감동적이고 훌륭한 영화', np.int64(1)), ('지루하고 별로였어 다시 안봐', np.int64(0))]
Dropout 예측: [('정말 감동적이고 훌륭한 영화', np.int64(1)), ('지루하고 별로였어 다시 안봐', np.int64(0))]
 BiLSTM 예측: [('정말 감동적이고 훌륭한 영화', np.int64(1)), ('지루하고 별로였어 다시 안봐', np.int64(0))]
